# DML from Core Perspective

In [2]:
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship
from sqlalchemy import create_engine, MetaData, insert
from sqlalchemy import Table, Column, Integer, String, ForeignKey

In [3]:
# This object structure will be at the center of most operations we perform with both Core and ORM going forward
metadata_obj = MetaData()
metadata_obj

MetaData()

In [4]:
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)
user_table

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [5]:
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)
address_table

Table('address', MetaData(), Column('id', Integer(), table=<address>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('user_account.id'), table=<address>, nullable=False), Column('email_address', String(), table=<address>, nullable=False), schema=None)

In [6]:
# Emit CREATE TABLE statements, or DDL, to our SQLite database so that we can insert and query data from them
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)
metadata_obj.create_all(engine)

2024-08-13 20:34:14,350 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-13 20:34:14,350 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-08-13 20:34:14,350 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-13 20:34:14,355 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-08-13 20:34:14,355 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-13 20:34:14,355 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-08-13 20:34:14,357 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-13 20:34:14,357 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-08-13 20:34:14,357 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-13 20:34:14,357 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-08-13 20:34:14,357 INFO sqlalchemy.engine.Engine [no key 0.00041s] ()
2024-08-13 20:34:14,357 INFO sqlalchemy.engine.Engine 
C

## Insert

In [7]:
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
stmt #  instance of Insert

In [8]:
compiled = stmt.compile()
compiled, compiled.params

(<sqlalchemy.sql.compiler.StrSQLCompiler at 0x191fa3e13d0>,
 {'name': 'spongebob', 'fullname': 'Spongebob Squarepants'})

In [9]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2024-08-13 20:34:14,395 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-13 20:34:14,398 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-08-13 20:34:14,399 INFO sqlalchemy.engine.Engine [generated in 0.00176s] ('spongebob', 'Spongebob Squarepants')
2024-08-13 20:34:14,400 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
# CursorResult.inserted_primary_key returns a tuple because a primary key may contain multiple columns.
# This is known as a composite primary key.
result.inserted_primary_key

(1,)

In [11]:
# If we don’t actually use Insert.values() and just print out an “empty” statement,
# we get an INSERT for every column in the table:
print(insert(user_table))

INSERT INTO user_account (id, name, fullname) VALUES (:id, :name, :fullname)


In [12]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2024-08-13 20:34:14,435 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-13 20:34:14,435 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-08-13 20:34:14,435 INFO sqlalchemy.engine.Engine [generated in 0.00145s] [('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star')]
2024-08-13 20:34:14,435 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
# Tricky feature
from sqlalchemy import select, bindparam
scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {
                "username": "spongebob",
                "email_address": "spongebob@sqlalchemy.org",
            },
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

2024-08-13 20:34:14,450 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-13 20:34:14,450 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2024-08-13 20:34:14,450 INFO sqlalchemy.engine.Engine [generated in 0.00128s] [('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org')]
2024-08-13 20:34:14,450 INFO sqlalchemy.engine.Engine COMMIT


In [14]:
# Insert returning
insert_stmt = insert(address_table).returning(
    address_table.c.id, address_table.c.email_address
)
print(insert_stmt)

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


In [15]:
# Insert from select
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


## Select

In [16]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [17]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2024-08-13 20:44:56,006 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-13 20:44:56,007 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-08-13 20:44:56,008 INFO sqlalchemy.engine.Engine [generated in 0.00216s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2024-08-13 20:44:56,010 INFO sqlalchemy.engine.Engine ROLLBACK
